In [1]:
from telethon.sync import TelegramClient
import asyncio
import pandas as pd
import nest_asyncio
from telethon.tl.types import MessageMediaPoll
from telethon.tl.types import MessageReactions
import re
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm 
nest_asyncio.apply()

In [3]:
async def get_messages():
    message_data = []
    async with TelegramClient('anon', api_id, api_hash) as client:
        # Get the total number of messages to fetch
        total_messages = 7000

        # Create the progress bar
        progress_bar = tqdm(total=total_messages, desc='Fetching Messages', unit='message')

        async for message in client.iter_messages('mothershipsg', limit=7000):
            # Check if the message is a poll
            poll_question = poll_choices = None
            if isinstance(message.media, MessageMediaPoll):
                poll_question = message.media.poll.question
                poll_choices = [choice.text for choice in message.media.poll.answers]
            
            # Extract the reaction details
            reaction_data = []
            if message.reactions and isinstance(message.reactions, MessageReactions):
                for reaction in message.reactions.results:
                    emoticon = reaction.reaction.emoticon
                    count = reaction.count
                    reaction_data.append({emoticon: count})

            # Each message is a row of data
            row = {
                'message_id': message.id,
                'date': message.date,
                'poll_question': poll_question,
                'text': message.text,
                'reactions': reaction_data,
            }
            # Add the row to the list
            message_data.append(row)

            # Update the progress bar
            progress_bar.update()

    # Close the progress bar after the loop is finished
    progress_bar.close()

    # Convert the list of dictionaries into a DataFrame
    df = pd.DataFrame(message_data)
    return df

# Running the async function using asyncio.run
new_loop = asyncio.new_event_loop()
asyncio.set_event_loop(new_loop)
df = asyncio.run(get_messages())

Fetching Messages: 100%|██████████| 7000/7000 [01:10<00:00, 99.66message/s] 


In [4]:
#Removing poll messages
df = df[df['poll_question'].isnull()]
df = df.drop('poll_question', axis =1)

In [5]:
#removing messages with no text
df = df[df['text'] != '']

In [6]:
#There are very litte reactions when Telegram first launched reactions so I will filter out the earlier messages
df = df[df['date'] > '2022-02']

In [7]:
df['main_text'] = df['text'].str.split('\n')

In [15]:
df['url'] = df['text'].str.extract('(http.*)')[0]

In [16]:
df = df[df['url'].notna()]

In [17]:
df = df[(df['main_text'].apply(len) == 5) | (df['main_text'].apply(len) == 7) | (df['main_text'].apply(len) == 9) | (df['main_text'].apply(len) == 8)]

In [18]:
def join_except_last_three(strings_list):
    if isinstance(strings_list, list) and len(strings_list) >= 3:
        return ' '.join(strings_list[:-3]).strip()
    elif isinstance(strings_list, list):
        return ' '.join(strings_list).strip()
    else:
        return ''

# Apply the custom function to create a new column
df['joined_text'] = df['main_text'].apply(join_except_last_three)

In [19]:
df.sort_values(by='joined_text', ascending=True)

,message_id,date,poll_question,text,reactions,main_text,url,joined_text
5931,12053,2022-03-04 08:28:56+00:00,None,"""I don’t want Ukraine’s history to be a legend...","[{'😢': 405}, {'👍': 164}, {'❤': 45}, {'👎': 24},...","[""I don’t want Ukraine’s history to be a legen...",https://bit.ly/35uHMm7,"""I don’t want Ukraine’s history to be a legend..."
2581,15453,2022-12-19 13:31:00+00:00,None,"""If the owners are unable to fulfil the minim...","[{'👎': 647}, {'👍': 428}, {'😱': 23}, {'😁': 13},...","[""If the owners are unable to fulfil the mini...",http://bit.ly/3HQPzvc,"""If the owners are unable to fulfil the minim..."
1121,16934,2023-05-01 09:32:34+00:00,None,"""So relax folks, there is really no need to hy...","[{'👎': 818}, {'👍': 104}, {'❤': 23}, {'😁': 16},...","[""So relax folks, there is really no need to h...",https://t.me/mothershipsg)__,"""So relax folks, there is really no need to hy..."
1313,16738,2023-04-14 05:02:05+00:00,None,"""Success 9"", the S’pore-registered vessel whic...","[{'😱': 1062}, {'😢': 145}, {'😁': 22}, {'🌚': 18}...","[""Success 9"", the S’pore-registered vessel whi...",http://bit.ly/43JStdZ,"""Success 9"", the S’pore-registered vessel whic..."
3512,14514,2022-09-24 10:25:05+00:00,None,"""Thanks also, of course, to the inspirational ...","[{'😁': 956}, {'😱': 135}, {'👎': 53}, {'👍': 35},...","[""Thanks also, of course, to the inspirational...",https://bit.ly/3DOzYdC,"""Thanks also, of course, to the inspirational ..."
...,...,...,...,...,...,...,...,...
2487,15547,2022-12-28 07:43:01+00:00,None,🫣 A 45-year-old tourist from New Zealand fell ...,"[{'😱': 826}, {'😢': 145}, {'👍': 47}, {'😁': 40},...",[🫣 A 45-year-old tourist from New Zealand fell...,https://bit.ly/3G5Ospc,🫣 A 45-year-old tourist from New Zealand fell ...
4046,13973,2022-08-07 11:46:21+00:00,None,🫣 A man in M'sia confessed to eating a puppy h...,"[{'👎': 1962}, {'😱': 595}, {'😢': 156}, {'🔥': 11...",[🫣 A man in M'sia confessed to eating a puppy ...,https://bit.ly/3Q92Y2Y,🫣 A man in M'sia confessed to eating a puppy h...
4230,13786,2022-07-22 12:24:26+00:00,None,🫣 Here’s a no-holds-barred review of KFC Korea...,"[{'🎉': 156}, {'👎': 126}, {'😁': 71}, {'😱': 55},...",[🫣 Here’s a no-holds-barred review of KFC Kore...,https://bit.ly/3OslqSs,🫣 Here’s a no-holds-barred review of KFC Korea...
2688,15346,2022-12-09 11:30:08+00:00,None,🫣 S'porean influencers Noah Yap & Chiou Huey m...,"[{'👎': 738}, {'😁': 169}, {'😱': 114}, {'👍': 46}...",[🫣 S'porean influencers Noah Yap & Chiou Huey ...,http://bit.ly/3hcVbFe,🫣 S'porean influencers Noah Yap & Chiou Huey m...


In [20]:
# Convert from UTC to SGT (which is 'Asia/Singapore' in pytz)
df['date'] = df['date'].dt.tz_convert('Asia/Singapore')


# Convert datetime to string
df['time'] = df['date'].dt.strftime('%H:%M')
df['date'] = df['date'].dt.strftime('%Y-%m-%d')

In [21]:
def extract_shortened_url(url):
    pattern = r'\bhttps?://bit\.ly/\w+\b'
    match = re.search(pattern, url)
    if match:
        return match.group()
    else:
        return url
    
df['url'] = df['url'].apply(extract_shortened_url)

In [22]:
df = df[~df['url'].str.len().isin([28, 26, 24, 17, 196, 138, 55, 32, 29, 30, 206, 124, 58, 45])]

In [24]:
df.to_excel('Dataset/mothershipsg.xlsx', index=False)

## HERE

In [2]:
df = pd.read_excel('Dataset/mothershipsg.xlsx')

In [3]:
df

,message_id,date,text,reactions,main_text,url,joined_text,time
0,18063,2023-07-25,🥺 A cat is balding because his friend kept lic...,"[{'😁': 576}, {'❤': 140}, {'🤩': 17}, {'😢': 8}, ...",['🥺 A cat is balding because his friend kept l...,https://bit.ly/3q4COXw,🥺 A cat is balding because his friend kept lic...,23:01
1,18062,2023-07-25,😱 A man in China attempted to blackmail a rest...,"[{'😱': 325}, {'😁': 71}, {'👎': 46}, {'🌚': 18}, ...",['😱 A man in China attempted to blackmail a re...,https://bit.ly/3DtL4n2,😱 A man in China attempted to blackmail a rest...,22:00
2,18061,2023-07-25,🌕 2 supermoons will be visible across S'pore o...,"[{'🌚': 267}, {'❤': 64}, {'🤩': 27}, {'👍': 18}, ...","[""🌕 2 supermoons will be visible across S'pore...",https://bit.ly/3OrKJGH,🌕 2 supermoons will be visible across S'pore o...,21:02
3,18060,2023-07-25,"☝️ The owner of a peacock, which attacked a 3-...","[{'👎': 608}, {'😱': 97}, {'👍': 52}, {'😢': 26}, ...","['☝️ The owner of a peacock, which attacked a ...",https://bit.ly/3KhngGH,"☝️ The owner of a peacock, which attacked a 3-...",20:01
4,18059,2023-07-25,🙃 Prices of Spotify premium plans are going up...,"[{'👎': 1341}, {'😢': 170}, {'👍': 18}, {'😱': 14}...",['🙃 Prices of Spotify premium plans are going ...,https://bit.ly/3DsjOFn,🙃 Prices of Spotify premium plans are going up...,19:01
...,...,...,...,...,...,...,...,...
5933,11691,2022-02-01,🧧 Would you dress up as a fortune cat for Chin...,"[{'👍': 307}, {'😁': 95}, {'👎': 28}, {'❤': 19}, ...",['🧧 Would you dress up as a fortune cat for Ch...,https://bit.ly/3KSHq8C,🧧 Would you dress up as a fortune cat for Chin...,16:30
5934,11690,2022-02-01,🎬 Our writer got to watch 'Ah Girls Go Army’ o...,"[{'👎': 364}, {'👍': 61}, {'😁': 23}, {'😱': 15}, ...","[""🎬 Our writer got to watch 'Ah Girls Go Army’...",https://bit.ly/3LavQWK,🎬 Our writer got to watch 'Ah Girls Go Army’ o...,15:30
5935,11689,2022-02-01,"A total of 62 men & women, aged between 25 to ...","[{'🎉': 176}, {'👍': 71}, {'😱': 35}, {'👎': 20}, ...","['A total of 62 men & women, aged between 25 t...",https://bit.ly/3oAOX2D,"A total of 62 men & women, aged between 25 to ...",13:31
5936,11688,2022-02-01,"[Ad] Just in time for Chinese New Year, StarHu...","[{'👎': 126}, {'👍': 45}, {'😱': 11}, {'🔥': 8}, {...","['[Ad] Just in time for Chinese New Year, Star...",https://bit.ly/3APERzr,"[Ad] Just in time for Chinese New Year, StarHu...",12:00


In [4]:
def get_web_text(link):
    try:
        response = requests.get(link)
        soup = BeautifulSoup(response.text, 'lxml')
        para = soup.find_all("div", class_="content-article-wrap")
        p_contents = re.findall(r'<p>(.*?)</p>', str(para), re.DOTALL)
        for i in range(len(p_contents)):
            p_contents[i] = re.sub(r'<[^>]+>', '', p_contents[i])
        filtered_list = list(filter(None, p_contents))
        filtered_list_no_whitespace = [s.strip() for s in filtered_list]
        result_string = ' '.join(filtered_list_no_whitespace)
        return result_string.replace('Follow us on Telegram for the latest updates: https://t.me/mothershipsg ', '')
    except (requests.RequestException, ValueError) as e:
        print(f"Error occurred for URL: {link}")
        print(e)
        return None

In [5]:
get_web_text('https://bit.ly/3LavQWK')

'As someone who was forced to be a hermit almost seven days a week thanks to Covid-19, going out for events have been such a treat. It\'s like an extended break on top of my hour-long lunchtime, if you will. So when my editor arrowed asked me if I could go for the media preview of Jack Neo\'s "Ah Girls Go Army" (AGGA), I said most enthusiastically, "Ok can". Here\'s the official synopsis: "WHAT IF, in the unforeseen future, when there are not enough boys in Singapore? If Girls have to serve National Service also, what will happen? The story will focus on the first batch of female recruits, a bunch of Gen Z teens and youngsters with different backgrounds and education. As they trained under the fierce leadership of Sergeant Chow and Lieutenant Roxanne, they find themselves being pushed to the limit of their potential. Together, they overcome hardship and initial resistance to serve NS and discover newfound abilities, using it to solve and mend relationships in their personal life as wel

In [6]:
import warnings
import urllib3

# Filter and ignore the InsecureRequestWarning
warnings.filterwarnings("ignore", category=urllib3.exceptions.InsecureRequestWarning)

In [33]:
from concurrent.futures import ThreadPoolExecutor
def process_links_with_multithreading(links):
    with ThreadPoolExecutor(max_workers=12) as executor:  # Set the number of threads you want to use
        # Use list comprehension to submit tasks to the executor
        results = [executor.submit(get_web_text, link) for link in links]
        
        # Get the results when the tasks are completed
        return [future.result() for future in results]

In [34]:
if __name__ == "__main__":
    # Replace links_list with the list of unique links in the 'url' column of df
    links_list = df['url'].unique().tolist()

    # Initialize the progress bar
    progress_bar = tqdm(total=len(links_list), desc='Processing URLs')

    def update_progress(future):
        # This function is called whenever a thread completes its task
        progress_bar.update(1)

    # Process links using multi-threading
    with ThreadPoolExecutor() as executor:
        # Submit tasks and use update_progress as a callback when tasks complete
        futures = [executor.submit(get_web_text, link) for link in links_list]
        for future in futures:
            future.add_done_callback(update_progress)

        # Get the results when the tasks are completed
        results = [future.result() for future in futures]

    # Create a dictionary to map URLs to their corresponding web content
    url_to_web_content = {link: content for link, content in zip(links_list, results)}

    # Update the 'WebText' column in the DataFrame with the web content
    df['web_text'] = df['url'].map(url_to_web_content)

    # Close the progress bar
    progress_bar.close()

Processing URLs:   6%|▋         | 371/5737 [00:23<05:08, 17.39it/s]

Error occurred for URL: https://bit.ly/3PoE02h
HTTPSConnectionPool(host='www.crateandbarrel.com.sg', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C632C7CB80>: Failed to establish a new connection: [WinError 10013] An attempt was made to access a socket in a way forbidden by its access permissions'))


Processing URLs:  30%|███       | 1730/5737 [01:43<02:50, 23.47it/s]

Error occurred for URL: http://bit.ly/3xorXaX
HTTPSConnectionPool(host='voting.youthactionplan.sg', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C63C63C6D0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))


Processing URLs:  41%|████▏     | 2375/5737 [02:22<02:05, 26.74it/s]

Error occurred for URL: http://bit.ly/3BtbFQi
HTTPSConnectionPool(host='mothership.sg', port=443): Max retries exceeded with url: /2022/12/sim-ann-leong-mun-wai-bto/?utm_source=tele+&utm_medium=referral+&utm_campaign=MS+ (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_BAD_RECORD_MAC] sslv3 alert bad record mac (_ssl.c:2548)')))


Processing URLs:  70%|███████   | 4035/5737 [04:10<01:21, 20.82it/s]

Error occurred for URL: https://bit.ly/3MUc816
HTTPSConnectionPool(host='sco.com.sg', port=443): Max retries exceeded with url: /scmc/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)')))


Processing URLs: 100%|█████████▉| 5735/5737 [05:55<00:00, 24.15it/s]C:\Users\user\AppData\Local\Temp\ipykernel_10832\2761072053.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['web_text'] = df['url'].map(url_to_web_content)
Processing URLs: 100%|██████████| 5737/5737 [05:57<00:00, 16.06it/s]

Error occurred for URL: https://bit.ly/3AMcMt9
HTTPSConnectionPool(host='posb.hyz.sqkii.com', port=443): Max retries exceeded with url: /?dclid=CLmKjISkqoADFUuoZgId1v0IEw (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C63302B4F0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))


In [35]:
df['web_text'] = df['web_text'].str.strip()

C:\Users\user\AppData\Local\Temp\ipykernel_10832\746556694.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['web_text'] = df['web_text'].str.strip()


In [36]:
df = df[df['web_text'] != '']

In [40]:
df.to_excel('Dataset/mothershipsg.xlsx', index=False)